In [1]:
import pprint
from duckduckgo_search import DDGS
from typing import List, Dict, Any, Optional
import logging
from autogen import ConversableAgent

# Parameters

In [2]:
with open("../secret/openai.txt", "r") as f:
    OPENAI_API_KEY = f.read()

In [3]:
MODEL = "gpt-4o-mini"

In [ ]:
global current_search_result

# Tool

In [4]:
def duckduckgo_search(search_term: str, max_results: int = 5) -> Optional[List[Dict[str, Any]]]:
    """
    Perform a DuckDuckGo search and return a list of results.

    Args:
        search_term (str): The term to search for.
        max_results (int, optional): The maximum number of search results to return. Defaults to 5.

    Returns:
        Optional[List[Dict[str, Any]]]: A list of dictionaries containing the search results, or None if an error occurs.
    """
    try:
        # Perform the search using DuckDuckGo
        results = DDGS().text(search_term, max_results=max_results)
        
        # Return the search results
        return results
    
    except Exception as e:
        logging.error(f"An error occurred during the DuckDuckGo search: {e}")
        return None

In [5]:
query = "quantum computing"

result = duckduckgo_search(query, max_results=2)

In [6]:
pprint.pprint(result)

[{'body': 'Learn about quantum computing, a computer that exploits quantum '
          'mechanical phenomena to perform calculations faster than classical '
          'computers. Explore the history, algorithms, technologies, and '
          'applications of quantum computing, as well as the obstacles and '
          'opportunities it faces.',
  'href': 'https://en.wikipedia.org/wiki/Quantum_computing',
  'title': 'Quantum computing - Wikipedia'},
 {'body': 'Quantum computing is an emergent field of cutting-edge computer '
          'science harnessing the unique qualities of quantum mechanics to '
          'solve problems beyond the ability of even the most powerful '
          'classical computers. The field of quantum computing contains a '
          'range of disciplines, including quantum hardware and quantum '
          'algorithms.',
  'href': 'https://www.ibm.com/topics/quantum-computing',
  'title': 'What Is Quantum Computing? | IBM'}]


# Agents

## Roles

In [8]:
assistant = ConversableAgent(
    name="Assistant_Search_Engine", # must comply with naming rules
    system_message="You are a helpful assistant. "
    "You will help search using a search engine. You will suggest one best search term. "
    "Follow the user's instruction on number of search websites to return. "
    "If the number is not spcified, use max_results=5 as a default. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{
        "model": MODEL, 
        "api_key": OPENAI_API_KEY,}]},
    max_consecutive_auto_reply=1,
    human_input_mode="NEVER",
)

user_proxy = ConversableAgent(
    name="User Proxy",
    llm_config=None,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    max_consecutive_auto_reply=3
)

## Tool registration

In [9]:
assistant.register_for_llm(name="duckduckgo_search",
                           description="search engine from DuckDuckGO"
                           )(duckduckgo_search)

user_proxy.register_for_execution(name="duckduckgo_search",
                                 )(duckduckgo_search)

<function __main__.duckduckgo_search(search_term: str, max_results: int = 5) -> Optional[List[Dict[str, Any]]]>

In [10]:
assistant.llm_config['tools']

[{'type': 'function',
  'function': {'description': 'search engine from DuckDuckGO',
   'name': 'duckduckgo_search',
   'parameters': {'type': 'object',
    'properties': {'search_term': {'type': 'string',
      'description': 'search_term'},
     'max_results': {'type': 'integer',
      'default': 5,
      'description': 'max_results'}},
    'required': ['search_term']}}}]

We assigned `register_for_execution` to `user_proxy` because it's two-agent conversation. They take turn.
1. `user_proxy` asks for search
2. `assistant` suggests how to 'call' (or search)
3. `user_proxy` does the actual search and gets results

# Conversation

In [11]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="search about quantum computing"
)

User Proxy (to Assistant_Search_Engine):

search about quantum computing

--------------------------------------------------------------------------------
Assistant_Search_Engine (to User Proxy):

***** Suggested tool call (call_V6xkt01UMKXrzPvMKMkTxee7): duckduckgo_search *****
Arguments: 
{"search_term":"quantum computing","max_results":5}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION duckduckgo_search...
User Proxy (to Assistant_Search_Engine):

User Proxy (to Assistant_Search_Engine):

***** Response from calling tool (call_V6xkt01UMKXrzPvMKMkTxee7) *****
[{"title": "Quantum computing - Wikipedia", "href": "https://en.wikipedia.org/wiki/Quantum_computing", "body": "Learn about the origins, concepts and developments of quantum computing, a field that exploits quantum mechanical phenomena for faster and more efficient calculations. Explor

In [12]:
chat_result.chat_history

[{'content': 'search about quantum computing', 'role': 'assistant'},
 {'tool_calls': [{'id': 'call_V6xkt01UMKXrzPvMKMkTxee7',
    'function': {'arguments': '{"search_term":"quantum computing","max_results":5}',
     'name': 'duckduckgo_search'},
    'type': 'function'}],
  'content': None,
  'role': 'assistant'},
 {'content': '[{"title": "Quantum computing - Wikipedia", "href": "https://en.wikipedia.org/wiki/Quantum_computing", "body": "Learn about the origins, concepts and developments of quantum computing, a field that exploits quantum mechanical phenomena for faster and more efficient calculations. Explore the applications, limitations and controversies of quantum algorithms, qubits, encryption and simulation."}, {"title": "What Is Quantum Computing? | IBM", "href": "https://www.ibm.com/topics/quantum-computing", "body": "Quantum computing is an emergent field of cutting-edge computer science harnessing the unique qualities of quantum mechanics to solve problems beyond the ability o

In [13]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 3.51e-05,
  'gpt-4o-mini-2024-07-18': {'cost': 3.51e-05,
   'prompt_tokens': 134,
   'completion_tokens': 25,
   'total_tokens': 159}},
 'usage_excluding_cached_inference': {'total_cost': 0}}